In [ ]:
#!/usr/bin/env python
# coding: utf-8



import numpy as np
from math import cos, sin, radians, sqrt, pi, atan, tan




class Elemento():
    def __init__(self,elemento, base, altura, moduloElastico, coordenada_xi, coordenada_yi, coordenada_xf, coordenada_yf, conexion):
        
        self.Elem = elemento
        self.b = base
        self.h = altura
        self.E = moduloElastico
        self.cxi = coordenada_xi
        self.cyi = coordenada_yi
        self.cxf = coordenada_xf
        self.cyf = coordenada_yf
        self.conex = conexion
        # Propiedades del elemento
        self.A = self.Area()
        self.I = self.Inercia()
        self.L = self.Longitud()
        self.ang = self.Angulo()
        self.l = self.coseno()
        self.m = self.seno()

    # Propiedades del elemento

    def Area(self):
        return self.b * self.h
    def Inercia(self):
        return (self.b * self.h**3) / 12
    def Longitud(self):
        return sqrt((self.cxf - self.cxi)**2 + (self.cyf - self.cyi)**2)
    def Angulo(self):
        b = (self.cxf - self.cxi)
        h = (self.cyf - self.cyi)
        try:
            angulo = atan(h/b)*180/pi
        except ZeroDivisionError:
            angulo = 90
        return angulo

    def seno(self):
        return sin(radians(self.ang))

    def coseno(self):
        return cos(radians(self.ang))        








E  = 217370.65

tbl_Elem = [
    
  # Elem, Ni, Nf, b, h, E,
    ["1", 1, 4, 25, 25, E],
    ["2", 2, 5, 25, 25, E],
    ["3", 3, 6, 25, 25, E],
    ["4", 4, 8, 25, 25, E],
    ["5", 5, 9, 25, 25, E],
    ["6", 6, 10, 25, 25, E],
    ["7", 4, 5, 25, 50, E],
     ["8", 5, 6, 25, 50, E],
     ["9", 6, 7, 25, 50, E],
    ["10", 8, 9, 25, 50, E],
     ["11", 9, 10, 25, 50, E],
     ["12", 10, 11, 25, 50, E],


    ]
    
tbl_Nod = [
    #Nodo, cx, cy
    [1, 0, 0],
    [2, 500, 0],
    [3, 900, 0],
    [4, 0, 400],
    [5, 500, 400],
    [6, 900, 400],
    [7, 1050, 400],
    [8, 0, 700],
    [9, 500, 700],
    [10, 900, 700],
    [11, 1050, 700],

    ]
        #El, Ni, Nf
#print(nodos[:])


tbl_Fuer = [
    #Nod, fx, fy, Mz
    # [3, 700, 0, 0],

  
    ]

tbl_Fuer_W = [
    #Elem, Li,Lf, Wi, Wf
    ["7", 0, 500, -7, -7],
    ["8",0, 400,  -7, -7,],
    ["9", 0, 150, -7, -7],
    ["10",0, 500,  -7, -7,],
    ["11", 0, 400, -7, -7],
    ["12",0, 150,  -7, -7,],

    ]

tbl_Rest = [
        #Tipo de Apoyo, Nodo
        # Simple
        # Articulado
        # Empotrado
    ["Empotrado", 1],
    ["Empotrado", 2,],
    ["Empotrado", 3,],
    ]
"""====Diccionarios===="""
# ==============Diccionarios de elementos==================#

diccionarioE = {}

for i in range(len(tbl_Elem)):
    Elem = tbl_Elem[i][0]
    Ni = tbl_Elem[i][1]
    Nf = tbl_Elem[i][2]
    b = tbl_Elem[i][3]
    h = tbl_Elem[i][4]
    E = tbl_Elem[i][5]   
    diccionarioE[Elem] = [Ni, Nf, b, h, E]

# ==============Diccionarios de nodos==================#
    
diccionarioN = {}

for i in range(len(tbl_Nod)):
    Nod = tbl_Nod[i][0]
    cx = tbl_Nod[i][1]
    cy = tbl_Nod[i][2]
    diccionarioN[Nod] = [cx, cy]
    




elementos = [] # Almacena todos los elementos con sus propiedades

def ext_elem(): # Elementos fuera del analisis

    for i in range(len(tbl_Elem)):
        Ni = tbl_Elem[i][1]
        Nf = tbl_Elem[i][2]
        b = tbl_Elem[i][3]
        h = tbl_Elem[i][4]
        Ele = tbl_Elem[i][0]    
        conexion = [Ele, Ni, Nf]
        xi = diccionarioN[Ni][0]
        yi = diccionarioN[Ni][1]
        xf = diccionarioN[Nf][0]
        yf = diccionarioN[Nf][1]     
        barra = Elemento(Ele,b, h, E, xi, yi, xf, yf, conexion)
        elementos.append(barra) # Almacena elemento
    return elementos


elementos = ext_elem()
        






# # SUB-DIVISIONES



sub = 10
coordenadas = []
coor_sub_elementos = []
cheq = []
for i in range(len(tbl_Elem)):
    coor_p = []
    Elemento = elementos[i]
    L = Elemento.L
    L_unitaria = L / sub
    Ni = Elemento.conex[1]
    Nf = Elemento.conex[2]
    cxi = Elemento.cxi
    cyi = Elemento.cyi
    cxf = Elemento.cxf
    cyf = Elemento.cyf
    angulo = Elemento.ang
    cont = 0
    cx = 0
    cy = 0
    for j in range(sub+1):
        if angulo == 0: 
            cy = Elemento.cyi
            cx = Elemento.cxi + L_unitaria*j
            coor_p.append([cx,cy])
            
        if cont < sub:  
            if angulo == 0: 
                cont += 1 
                cy = Elemento.cyi
                cx = Elemento.cxi + L_unitaria*j
                cheq.append([cx,cy])
                #if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                    #coordenadas.append([cx,cy])
                    
        # Elementos verticales
        if angulo == 90:
            cx = Elemento.cxi
            cy = Elemento.cyi + L_unitaria*j            
            cheq.append([cx,cy])
            coor_p.append([cx,cy])
     
        # Elementos inclinados crecientes
        
        elif angulo > 0 and angulo < 90:
            """estos fracmento forman l_uni en un triangulo rectangulo"""
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(angulo))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly_uni/tan(radians(angulo)) # Longitud del fracmento en x
            cx = Elemento.cxi + Lx_uni*j
            cy = Elemento.cyi + Ly_uni*j
            
            cheq.append([cx,cy])
            coor_p.append([cx,cy])

        # Elementos inclinados decrecientes
        elif angulo < 0:
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(angulo))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly_uni/tan(radians(angulo)) # Longitud del fracmento en x
            cx = Elemento.cxi - Lx_uni*j
            cy = Elemento.cyi - Ly_uni*j
            
            cheq.append([cx,cy])
            coor_p.append([cx,cy])

        if [cx, cy] not in coordenadas: # Se chequea si la coord ya esta incluida
                    coordenadas.append([cx,cy])
    coor_sub_elementos.append(coor_p)
#coor_sub_elementos




tbl_sub_nodos = []
dic_sub_nodos = {}
id_nodo = 0
for n in range(len(coordenadas)):
    cx, cy = coordenadas[n]
    id_nodo += 1
    tbl_sub_nodos.append([id_nodo, cx, cy])
    dic_sub_nodos[id_nodo] = [cx, cy]
#tbl_sub_nodos 

#dic_sub_nodos 







tbl_sub_elementos = []
dic_sub_elementos = {}
id_elementos = 0
for i in range(len(tbl_Elem)):
    Elemento = elementos[i]
    Elem = Elemento.Elem
    b = Elemento.b
    h = Elemento.h
    E = Elemento.E
    angulo = Elemento.ang
    sub_elementos_p = []
    for j in range(sub):
        id_elementos += 1
        
        ci = coor_sub_elementos[i][j]
        cf = coor_sub_elementos[i][j+1]

        for m in range(len(tbl_sub_nodos)):

            N = m+1   
            
            if dic_sub_nodos[N] == ci:
                Ni = N
            if dic_sub_nodos[N] == cf:
                Nf = N  
        tbl_sub_elementos.append([str(id_elementos), Ni, Nf, b, h, E])
        sub_elementos_p.append([str(id_elementos), Ni, Nf, b, h, E])
    sub_elementos_x_Elemento[Elem] = sub_elementos_p
    dic_sub_elementos[Elem] = sub_elementos_p          




class Sub_Elementos():
    def __init__(self,elemento, base, altura, moduloElastico, coordenada_xi, coordenada_yi, coordenada_xf, coordenada_yf, conexion):
        
        self.Elem = elemento
        self.b = base
        self.h = altura
        self.E = moduloElastico
        self.cxi = coordenada_xi
        self.cyi = coordenada_yi
        self.cxf = coordenada_xf
        self.cyf = coordenada_yf
        self.conex = conexion
        # Propiedades del elemento
        self.A = self.Area()
        self.I = self.Inercia()
        self.L = self.Longitud()
        self.ang = self.Angulo()
        self.l = self.coseno()
        self.m = self.seno()

    # Propiedades del elemento

    def Area(self):
        return self.b * self.h
    def Inercia(self):
        return (self.b * self.h**3) / 12
    def Longitud(self):
        return sqrt((self.cxf - self.cxi)**2 + (self.cyf - self.cyi)**2)
    def Angulo(self):
        b = (self.cxf - self.cxi)
        h = (self.cyf - self.cyi)
        try:
            angulo = atan(h/b)*180/pi
        except ZeroDivisionError:
            angulo = 90
        return angulo

    def seno(self):
        return sin(radians(self.ang))

    def coseno(self):
        return cos(radians(self.ang))        




sub_elementos = [] # Almacena todos los elementos con sus propiedades
for i in range(len(tbl_sub_elementos)):
    Ni = tbl_sub_elementos[i][1]
    Nf = tbl_sub_elementos[i][2]
    b = tbl_sub_elementos[i][3]
    h = tbl_sub_elementos[i][4]
    Ele = tbl_sub_elementos[i][0]    
    conexion = [Ele, Ni, Nf]
    xi = dic_sub_nodos[Ni][0]
    yi = dic_sub_nodos[Ni][1]
    xf = dic_sub_nodos[Nf][0]
    yf = dic_sub_nodos[Nf][1]     
    barra = Sub_Elementos(Ele,b, h, E, xi, yi, xf, yf, conexion)
    sub_elementos.append(barra) # Almacena elemento




# Obtener cargar externas para los sub elementos
tbl_sub_F_W = []

for i in range(len(tbl_Fuer_W)):
    Elemento = tbl_Fuer_W[i][0]
    wi = tbl_Fuer_W[i][3]
    wf = tbl_Fuer_W[i][4]
    L = 0
    
    for n in range(len(elementos)): # Extra la longitud del elemento sin sub...
        if Elemento == elementos[n].Elem:
            L = elementos[n].L

    for j in range(sub):
        
        Elem = dic_sub_elementos[Elemento][j][0]
   
        ################################################
        ##################################################
        # Carga distibuida rectangular
        #print(wi, wf)
        if wi == wf:
            
            L_unitaria = L/sub*(j+1)
            #print(L_unitaria)
            sub_W = wi
            tbl_sub_F_W.append([Elem, L/sub*j, L_unitaria,sub_W , sub_W])
            
            
        #########################################################
        ##############################################################
            
        # Carga distibuida triangular decreciente
        
        if  abs(wi) > 0  and abs(wf) == 0:
            
            W = wi
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        
        # Carga distibuida triangular creciente
        if  abs(wf) > 0  and abs(wi) == 0:
            
            W = wf
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal decreciente
        if  abs(wi) > abs(wf) and abs(wf) > 0:
            
            W = (wi - wf)
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L + wf
            sub_wf = (W * L_unitaria * (j+1))/L + wf
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal creciente
        if  abs(wi) < abs(wf) and abs(wi) > 0:
            
            W = (wf - wi)
            L_unitaria = L/sub
            sub_wi = wi + (W * L_unitaria * j) /L
            sub_wf = wi + (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])


#tbl_sub_F_W      


# # Consulta por el nodo real



def nodo_ireal(nodo):
    coordenadas_reales = diccionarioN[nodo]
    for i in range(len(tbl_sub_nodos)):
        sub_N = i+1
        if coordenadas_reales == dic_sub_nodos[sub_N]:
            Nod = sub_N
            return Nod




tbl_sub_Fuer = []

for j in range(len(tbl_Fuer)):
    Nod = tbl_Fuer[j][0]
    fx = tbl_Fuer[j][1]
    fy = tbl_Fuer[j][2]
    mz = tbl_Fuer[j][3]
    sub_N = nodo_ireal(Nod)
    tbl_sub_Fuer.append([sub_N, fx, fy, mz])
#tbl_sub_Fuer





tbl_sub_restricciones = []

for j in range(len(tbl_Rest)):
    N_rest = tbl_Rest[j][1]
    tipo = tbl_Rest[j][0]
    sub_N = nodo_ireal(N_rest)
    tbl_sub_restricciones.append([tipo, sub_N])
tbl_sub_restricciones




tbl_sub_elementos


tbl_sub_nodos





tbl_sub_Fuer


tbl_sub_F_W



tbl_sub_restricciones

nodo_ireal(3)
#sub_elementos_x_Elemento["8"]






